<a href="https://colab.research.google.com/github/rishavsen1/programming-assignment-0/blob/main/Assignment_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this assignment we are going to work with data that was downloaded using using the [yfinance](https://pypi.org/project/yfinance/) library. It containst daily closing prices of all symbols.

In [1]:
from multiprocessing   import Pool
from tqdm import *
import pandas as pd

In [2]:
#Question 1: Explain what the code below is trying to do. Describe each step.
data = pd.read_csv("http://www.nasdaqtrader.com/dynamic/SymDir/nasdaqtraded.txt", sep='|')
data_clean = data[data['Test Issue'] == 'N']
symbols = set(data_clean['NASDAQ Symbol'].tolist())
data = pd.read_csv("http://www.nasdaqtrader.com/dynamic/SymDir/otherlisted.txt", sep='|')
data_clean = data[data['Test Issue'] == 'N']
symbols = set(data_clean['NASDAQ Symbol'].tolist())


In [ ]:
#Describe your answer below.
#Answer 1: 

1. The NASDAQ Traded dataset is read, where the columns are delimited by "|", and  is stored in "data" variable.<br>
2. "data_clean" stores the data where "Test Issues" are "N".<br>
3. A set of all the NASDAQ symbols is stored in "symbols"<br>
4. Similarly, The NASDAQ Other listed dataset is read, where the columns are delimited by "|", and  is stored in "data" variable.<br>
5. "data_clean" stores the data where "Test Issues" are "N".<br>
6. A set of all the NASDAQ other traded symbols is stored in "symbols"<br>


In [3]:
!wget  -O stocks.parquet https://www.dropbox.com/s/gzpi950gfdz9dsk/stocks.parquet?dl=1 # lets download the data

--2022-01-26 20:57:12--  https://www.dropbox.com/s/gzpi950gfdz9dsk/stocks.parquet?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.3.18, 2620:100:601b:18::a27d:812
Connecting to www.dropbox.com (www.dropbox.com)|162.125.3.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/dl/gzpi950gfdz9dsk/stocks.parquet [following]
--2022-01-26 20:57:12--  https://www.dropbox.com/s/dl/gzpi950gfdz9dsk/stocks.parquet
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc19d41c54389f66d2a7ca65135f.dl.dropboxusercontent.com/cd/0/get/Bei4Wb4IwNYLBvWMhWCRyDKjefTSAl5SKRraSv36C37CZ6uJYs4bu856ZCWnRnUl80itCaOUWPzfL-mHVo1a8gzy6gF5apw-SGK8oxwc_FXIs4szCGPiko9HzZHB3-zbkbLD9srAwbMja24fcui19-JY/file?dl=1# [following]
--2022-01-26 20:57:12--  https://uc19d41c54389f66d2a7ca65135f.dl.dropboxusercontent.com/cd/0/get/Bei4Wb4IwNYLBvWMhWCRyDKjefTSAl5SKRraSv36C37CZ6uJYs4bu856ZCWnRnUl80itCaOUWPzfL-mHVo

In [4]:
!ls stocks.parquet -l #downloaded file

-rw-r--r-- 1 root root 678640589 Jan 26 20:57 stocks.parquet


In [5]:
#Bonus question 1 - find on internet and write the command that will download the all data from the symbols list. 
symbols = pd.read_parquet('stocks.parquet', columns=["symbol"]) #all symbols
symbols_unique = symbols.drop_duplicates() #unique symbols

In [6]:
data=pd.read_parquet('stocks.parquet')

In [36]:
# Question 2 find top 20 best performer stocks - i.e. the stocks that provided the best return.
# if the stocks adj_close on the earliest date was X and the adj_close on the 2021-01-01 is y and the number of days elapsed between these dates is Z.
# Then return = Y/X/Z
# Sort the stocks by this return and find the top 20

In [ ]:
data.head(1) # Below shows a sample of the data.
#date - describes the day of the price
#open - the price at which stock opened
#high - what was the intraday high
#low - what is the intraday low
#close - what was the closing price
#adj_close - the price calculated after adjusting for all future stock splits
#dividends - what are the dividends that were paid on the day.
#stock_splits - if the stock was split
#symbol the symbol of the stock


,date,open,high,low,close,adj_close,volume,dividends,stock_splits,symbol,day,year,month,dayofweek
0,1962-01-02,0.131273,0.131273,0.124177,0.124177,0.045278,2480333.0,0.0,0.0,HPQ,2,1962,1,Tuesday


In [62]:
from datetime import datetime
import numpy as np

df = pd.read_parquet('stocks.parquet', columns=["date", "adj_close"])

d0 = datetime.strptime(df.iloc[0].date, '%Y-%m-%d').date()
X = df.iloc[0].adj_close
stock_start_ind = 0
ret = [np.nan for i in range(len(symbols_unique))]
count = 0
days = [1 for i in range(len(symbols_unique))]
start_adj_close = [np.nan for i in range(len(symbols_unique))]
end_adj_close = [np.nan for i in range(len(symbols_unique))]
df_return = symbols_unique.copy(deep=False)
for i, rows in symbols_unique[1:].iterrows():
  if i-stock_start_ind >=1.00007:  #to remove stocks with 1 day's performance recorded only
    Y = df.iloc[i-1].adj_close
    start_adj_close[count] = X
    end_adj_close[count] = Y
    d1 = datetime.strptime(df.iloc[i-1].date, '%Y-%m-%d').date()
    Z = d1 - d0
    Z = Z.days
    if Z == 0:
      Z=1
    days[count] = Z
    ret[count] = ((Y/X)/Z)
    d0 = datetime.strptime(df.iloc[i].date, '%Y-%m-%d').date()
    X = df.iloc[i].adj_close
  stock_start_ind = i-1
  count+=1

# to account for the last stock
if df.index[-1] - df.index[i] >= 1.00007:
  Y = df.iloc[-1].adj_close
  d1 = datetime.strptime(df.iloc[-1].date, '%Y-%m-%d').date()
  Z = d1 - d0
  Z = Z.days
  ret[len(ret)] = ((Y-X)/Z)
  
df_return['start_adj_close'] = start_adj_close
df_return['end_adj_close'] = end_adj_close
df_return['days'] = days  
df_return['ret'] = ret
pd.set_option('mode.use_inf_as_na', True)
# df_return.replace([np.inf, -np.inf], np.nan)
df_return.dropna(inplace=True)

df_return = df_return.sort_values(by='ret', ascending=False)
print("Best returning stocks are:\nSymbol \t Return")
df_return.head(20)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:25: RuntimeWarning: divide by zero encountered in double_scalars


Best returning stocks are:
Symbol 	 Return


,symbol,start_adj_close,end_adj_close,days,ret
1596422,NVO,2.275794e-09,111.629997,14851,3.302876e+06
2314327,NKE,3.775067e-06,167.580002,15000,2.959418e+03
3581760,DCI,9.812100e-06,57.939999,15260,3.869564e+02
6099850,TJX,2.857077e-05,74.290001,12603,2.063167e+02
1585886,WTRG,4.768529e-04,52.549999,15260,7.221605e+00
3329911,TTC,1.051490e-03,99.879997,15260,6.224708e+00
570633,USB,7.828808e-04,56.740002,17770,4.078555e+00
325655,MCD,3.923203e-03,268.239990,20264,3.374097e+00
7222467,PII,3.345692e-03,107.000000,12521,2.554223e+00
2621252,HD,1.823626e-02,404.089996,14706,1.506773e+00


In [ ]:
#Question 3: Create a copy of this notebook called assignment-aws and follow the instructions in readme to run the notebook on AWS compute instance.
# Compare performance of running natively on colab and colab+AWS

The AWS instance works comparatively faster than running natively on Colab. In my case, about 5 seconds faster, in performing all the steps.